# Introduction 

My project is about the spontanous occurance of atherosclerosis in birds. We have conducted a systematic review of all reported cases of spontanous atherosclerosis in birds. We are collecting the papers, 
species name and the order of the bird in each citation, whether athero was induced, whether the animals were captive and whether there is a mention of succeptibility or resistance to athero in each citation. 

This research take an evolutionary medicine approach to the question of occurance one of the most common and fatal "human" dieases in animals (finnlayson 1962). Although the appearance of atherosclerosis in birds has not been explored in literature, other comparative studies have taken this approach for other diseases. For instance, blumstein et al. (2015), have studies the capture myopathy in hooved mammals. Abbeglen et al. (2015) have found a mechanism for cancer resistence in elephants in a similar comparative approach.
The succeptibility or resistence found in parituclar avian groups can point to the inherent evolutionary causes of athero in humans, which can provide valuable insight into how evolution has shaped the factors that has increased humas's scucceptibility to atherosclerosis.

The aim of the project is to make ancestral reconstruction of succeptibility to atherosclerosis in birds. I then want to visualize the phylogenetic map of spontanouse atherosclerosis in birds and determine whether the differences in occurance of athero in different avian orders represent inherent succeptibility or resistance in particular bird orders. 

Finally I will analyze the relationship between various potential factors such as body mass, captivity and 
average flight altitude, and ocrrurance of spontanouse atherosclerosis in birds with pgls models. 


# Code and Steps

# 1
***First, I needed to know the orders that have occurance of atherosclerosis. Therefore, I have extracted the uniqe order in my dataset with the following shell command:***

```tail -n +2 athero.csv | cut -d "," -f 6 |sort|uniq```


***This is useful later on when identifying orders in the tree with atherosclerosis, output:***
```Anseriformes
Accipitriformes
Ciconiiformes
Columbiformes
Falconiformes
Galliformes
Gruiformes
Pelecaniformes
Psittaciformes
Struthioniformes```

# 2
***Then, I had to extract a representative tip_label from every order in master phylogeny of all birds, with nearly 10,000 species in it, the following function achieves that goal***

```
def make_tip_labels(filename="BLIOCPhyloMasterTaxPrumJarvis.csv"):
#this function will extract one representative tip label(species) from every bird oder in the master phylogeny file
    
 #tip label = list index 2, order = list index 13
    import csv
    import sys
    
    
    file_object=open(filename, "r", encoding= "ISO-8859-15")#open the file
    
    output=open("tip_label_reps.csv", "w")
    
    master_phylo=csv.reader(file_object)#read the file line by line 
    
    next(master_phylo,None)#skipping the header
    
    
   
    
    tip_label_dict={} #defining dictionary that will eventually have all tipl labels and orders
    
    specie="Struthio_camelus"#defining the first species for temp specie repo
    all_orders=["STRUTHIONIFORMES"]#defining the list of orders
    order="STRUTHIONIFORMES"# defning first order for temp order repo
    
    for line in master_phylo: #loop over every line of the master phylogeny file
        
       
        
        
        if line[13] in all_orders: #overwriting the order if the order changes in a line
            tip_label_dict[line[13]]=specie #populating the dictionary
            specie=line[2] #defining the new specie in particular line
            
        else:  
            
            
            
            orde=line[13]#updating the new temp order
            all_orders.append(line[13])#append the new order to the list of orders
            specie=line[2] #defining the new specie in particular line    
            
 
    
    return tip_label_dict

        
        ```

*** The above function returns the output as a dictionary which will be very useful in subsequent steps, output:***

```{'ACCIPITRIFORMES': 'Hieraaetus_weiskei',
 'ANSERIFORMES': 'Anas_eatoni',
 'APODIFORMES': 'Xiphorhynchus_elegans',
 'APTERYGIFORMES': 'Pyrilia_aurantiocephala',
 'BUCEROTIFORMES': 'Penelopides_samarensis',
 'CAPRIMULGIFORMES': 'Caprimulgus_heterurus',
 'CARIAMIFORMES': 'Cariama_cristata',
 'CASUARIIFORMES': 'Casuarius_unappendiculatus',
 'CHARADRIIFORMES': 'Gygis_microrhyncha',
 'CICONIIFORMES': 'Leptoptilos_crumeniferus',
 'COLIIFORMES': 'Urocolius_macrourus',
 'COLUMBIFORMES': 'Phapitreron_brunneiceps',
 'CORACIIFORMES': 'Todiramphus_recurvirostris',
 'CUCULIFORMES': 'Eudynamys_orientalis',
 'EURYPYGIFORMES': 'Chunga_burmeisteri',
 'FALCONIFORMES': 'Caracara_cheriway',
 'GALLIFORMES': 'Dendragapus_fuliginosus',
 'GAVIIFORMES': 'Eudyptes_moseleyi',
 'GRUIFORMES': 'Ducula_subflavescens',
 'MESITORNITHIFORMES': 'Mesitornis_unicolor',
 'MUSOPHAGIFORMES': 'Crinifer_zonurus',
 'OTIDIFORMES': 'Houbaropsis_bengalensis',
 'PASSERIFORMES': 'Cinclodes_olrogi',
 'PELECANIFORMES': 'Threskiornis_bernieri',
 'PHAETHONTIFORMES': 'Phaethon_rubricauda',
 'PHOENICOPTERIFORMES': 'Butorides_virescens',
 'PICIFORMES': 'Ramphastos_swainsonii',
 'PODICIPEDIFORMES': 'Aechmophorus_occidentalis',
 'PROCELLARIIFORMES': 'Thalassarche_carteri',
 'PSITTACIFORMES': 'Amazona_oratrix',
 'PTEROCLIDIFORMES': 'Pterocles_lichtensteinii',
 'RHEIFORMES': 'Rhea_americana',
 'SPHENISCIFORMES': 'Pelecanus_thagus',
 'STRIGIFORMES': 'Otus_siaoensis',
 'STRUTHIONIFORMES': 'Struthio_camelus',
 'SULIFORMES': 'Anhinga_novaehollandiae',
 'TINAMIFORMES': 'Apteryx_mantelli',
 'TROGONIFORMES': 'Trogon_aurantiiventris'} ```

# 3
*** Then, I have to make sure that the tip labels extracted from master phylogeny data do really exist in the bird tree file (.tre) to make sure that the tip labels are legitimate. The following function achieves that goal***

```
def check_tip_labels(tip_label_dict):
#this function will check if the extracted tip labels from make_tip_labels function exist in master bird tree file
#and out puts those tip labels in a csv file
    
    import csv
    import sys
    import re
    
    file_object=open("Hackett_Stage2_1K_MCCtree_TargetHeights.tre","r")#opens master bird tree
    master_bird_tree=file_object.read()#reads master bird tree as a string
    
    
    
    matches=0 #startig matches and misses counter
    misses=0 
    for values, keys in tip_label_dict.items():#loop goes over all key value pairs in the dictionary provided by m
    
        if re.match(pattern=keys, string=master_bird_tree)!="": #if search for the tip label is not empty, increase match by one
            matches=matches+1 #increases macthed by one
            print("Specie matched = "+keys) #prints the tip label that had been matched
        else:
            misses=misses+1 #otherwise, increases misses by one
    
    print("Matches = "+str(matches)+", Misses = "+str(misses)) #print the number of matches and misses

```    
   

***The output shows the name of tip labels (species) that had been matched along with the number of misses and matches, output:***

```
Specie matched = Urocolius_indicus
Specie matched = Tinamotis_ingoufi
Specie matched = Eleothreptus_anomalus
Specie matched = Merops_nubicus
Specie matched = Struthio_camelus
Specie matched = Aegotheles_albertisi
Specie matched = Rhynchortyx_cinctus
Specie matched = Sypheotides_indicus
Specie matched = Neomorphus_pucheranii
Specie matched = Fulica_cornuta
Specie matched = Uratelornis_chimaera
Specie matched = Eurypyga_helias
Specie matched = Dromaius_novaehollandiae
Specie matched = Phaethon_lepturus
Specie matched = Fratercula_cirrhata
Specie matched = Rhinopomastus_minor
Specie matched = Rhea_pennata
Specie matched = Monias_benschi
Specie matched = Ortyxelos_meiffrenii
Specie matched = Triclaria_malachitacea
Specie matched = Chelidoptera_tenebrosa
Specie matched = Falco_fasciinucha
Specie matched = Fregata_andrewsi
Specie matched = Spheniscus_mendiculus
Specie matched = Phalacrocorax_featherstoni
Specie matched = Gavia_adamsii
Specie matched = Psophia_viridis
Specie matched = Didunculus_strigirostris
Specie matched = Apteryx_haastii
Specie matched = Aechmophorus_clarkii
Specie matched = Oceanodroma_furcata
Specie matched = Sarcoramphus_papa
Specie matched = Sagittarius_serpentarius
Specie matched = Opisthocomus_hoazin
Specie matched = Harpactes_wardi
Specie matched = Mergus_merganser
Specie matched = Scopus_umbretta
Specie matched = Corythaeola_cristata
Specie matched = Selasphorus_ardens
Matches = 39, Misses = 0 ```

# 4
*** Then, I need to extract all tip labels that are not a representative of their order, This is necessary in order to prune down (shrink) the 10,000 bird species tree to few tip labels that represent each order, in order to analyze the data in the level of orders. The output is a large csv file close to the original master phylogeny (10,000 - represented orders). The following code achieves that.***

```
def make_non_occurance(tip_label_dict, filename="BLIOCPhyloMasterTaxPrumJarvis.csv"):
# this function extracts all non-representative species in order to drop those tip labels when working with R
    
    #tip label = list index[2]  order = list index [13]
    
    import csv
    import sys
    
    
    file_object=open(filename, "r", encoding= "ISO-8859-15")#open the file
    
    master_phylo=csv.reader(file_object)#read the file line by line 
    
    next(master_phylo,None)#skipping the header
    
    output=open("non_occurance_reps.csv", "w")#opening output file
    
      
    
    for line in master_phylo: #loop over every line of the master phylogeny file
       
        
       
    
        if line[2] not in dd.values(): #if the species is not in the list of representative species, it will be in output
            
            output.write(line[2]+"\r") #populating otput file with non-representative species 
            #"\r" is used because Rstudio handles line endings with "\r" better than "\n"
            
            
            
            ```

*** The output file is a large file, close to 10,000 lines, a small heading of the output is shown here:***
```
Rhea_pennata,
Casuarius_casuarius,
Casuarius_bennetti,
Dromaius_novaehollandiae,
Apteryx_australis,
Apteryx_owenii,
Apteryx_haastii,
Tinamus_tao,
Tinamus_solitarius,
Tinamus_osgoodi,
Tinamus_major,
Tinamus_guttatus,
Nothocercus_bonapartei,
Nothocercus_julius,
Nothocercus_nigrocapillus,
Crypturellus_berlepschi,
Crypturellus_cinereus,
Crypturellus_soui,
Crypturellus_ptaritepui,
Crypturellus_obsoletus,
Crypturellus_cinnamomeus,
...
```

# 5 
*** I also want to have a list of tip labels from orders that atherosclerosis have occurred in them for subsequent steps. The following code achieves that.***

```
def make_occurance(filename= "Dataset.csv"):
#this function will extract all orders of occurance of athero from a csv file and outputs the representative of each order that has been chosen by function 1
#this function is used to make a list of occurance order for later use in Rstudio
    
    import csv
    import sys
    
    file_object=open(filename, encoding= "ISO-8859-15") #openning the input file

    reader=file_object.readlines()[1:] #reads the input file
    
 
    
    output=open("occurance_species.csv", "w") #opens the output file foe later analysis in R
    
    
    
    
    function_1_output=make_tip_labels() #this defines the function 1 output but it is modified here for purpose of demonstration
    
    for line in reader:
        
       
        output.write(function_1_output[line]+"\r") #populates the output file, "\n" could also be used here, "\r" was used here because Rstudio handles "\r" better for later analysis
        
        
        print(function_1_output[line]) #for the purpose of demonstration the, output is shown here as well!
```


***output of tip labels with occurance of atherosclerosis:***
```
Ducula_subflavescens
Phapitreron_brunneiceps
Threskiornis_bernieri
Hieraaetus_weiskei
Caracara_cheriway
Amazona_oratrix
Struthio_camelus
Dendragapus_fuliginosus
Leptoptilos_crumeniferus
Anas_eatoni
```

# 6
*** The next step is working with the Rstudio to read the input csv files, and read the master tree file. Make a list of tip_labels that are not representative of an order and prune down the tree to all essential representative tips***


```
# installing packages and updating libriaries

install.packages(devtools)

library(devtools)
library(phytools)
library(psych)
library(ape)



#for reading all non-representative tip labels
nooccuranceallspecies<-read.csv("/Users/Mehdi/Documents/Classes/C177/Final project/non_occurance_reps.csv", header = TRUE)
length(nooccuranceallspecies)

#for reading the ,aster tree file 
complete.tree<-read.tree(file="/Users/Mehdi/Documents/Classes/C177/Final project/nature21074-s2/Data/Hackett_Stage2_1K_MCCtree_TargetHeights.tre")



#defining a list of all taxa to drop
taxa_to_drop=("")
i<-1

# this for loop iterates over all non represented species and makes a list of tip labels to drop in taxa_to_drop
for (j in nooccuranceallspecies$Species){
  taxa_to_drop[[i]]<-j
  i=i+1
  }

#for removing the unwanted tips from the tree file, or dropping the taxa_to_drop 
simple.tree<-drop.tip(complete.tree,taxa_to_drop)

#for plotting the tree
plotTree(simple.tree,type="fan",fsize=0.7,lwd=1)

```



***output is a bird tree with only representative species:***

<img src="./outlinefigs/Rplot.png" width="800" height="800" alt="Alt_name" title="Mytitle" align="center" />

# 7

*** Next I have to do ancestral state reconstruction for the discrete variable of occurance and non-occurance of atherosclerosis to examine the relationship of tip labels and internal nodes and the appearance of atherosclerosis***
*** this part still needs modification, the code here works, however, it still needs changes to represent the my data***

```
# update the libraries that are needed
library(phytools)
library(geiger)

#create a discrete variable, A=for presence of athero and B=for non-occurance of athero

y<-as.factor(c("A","A","A","B","B","A","A","B","A","B",
     "A","A","A","B","B","A","A","B","A","B",
     "A","A","A","B","B","A","A","B","A","B",
     "A","A","A","B","B","A","A","B"))
     
#create a variable that matches the tip labels with the state of occurance and non-occurance of athero
# this part will be modified for the presentation to reflect the data set.

x<-setNames(y,simple.tree$tip.label)

# defining tip labels 
tips <- simple.tree$tip.label

#write a csv file to add character data to tips, for later modifications of this code
write.csv("tipNames.txt", tips)


# single-rate model, where the rate change in charachters from one state to another is the same
fitER<-ace(x,simple.tree,model="ER",type="discrete")
fitER

#  model with the backward & forward rates between states are permitted to have different values
fitARD<-ace(x,simple.tree,model="ARD",type="discrete")
fitER

#creating the figure, making red and bue as factors that will match with A and B discrete variables
cols<-setNames(c("red","blue"),levels(x))

#creating tree with probabities of having athero vs no athero in tips and internal nodes all the way to the most ancient ancestor
plotTree(simple.tree,type="fan",fsize=0.6,ftype="i",lwd=1)

#adding pie chart in each node to represent the probability of being athero succeptible or resistant
nodelabels(node=1:simple.tree$Nnode+Ntip(simple.tree),
           pie=fitER$lik.anc,piecol=cols,cex=0.4)

tiplabels(pie=to.matrix(x[simple.tree$tip.label],levels(x)),
          piecol=cols,cex=0.3)

#adding legend to the figures
add.simmap.legend(colors=cols,prompt=FALSE,x=0.9*par()$usr[1],
                  y=0.8*par()$usr[3],fsize=0.8)
```


*** The output will show a pie chart for probability of having athero or no athero in that specific node in the bird tree:***


<img src="./outlinefigs/Rplot01.png" width="800" height="800" alt="Alt_name" title="Mytitle" align="center" />


# 8
***The next step will be pseudo code for finding the relationship between, species avregae body mass, average altitude of flight, LDHR receptors, longevity, ... with the occurance of atherosclerosis in diffrent bird orders. Following code is the foundation of such test ***

```
#updating libraries
library(ape)
library(nlme)
library(geiger)

#reading the data set file
datos<-read.csv("data.csv",header=TRUE,row.names=1)

# gls model, examining the relationship between body mass and athero
modelo1<-gls(datos$body.mass, data=datos, correlation=bm)
summary(modelo1)

# pgls mode, examining the relationship between longevity and athero
modelo4<-pgls(datos$lengevity, data=comp.data)
summary(modelo4)
```

*** a typical output might look like this:***
```
## Generalized least squares fit by REML
##   Model: Lnote ~ Lnalt 
##   Data: datos 
##         AIC     BIC   logLik
##   -68.33596 -64.034 37.16798
## 
## Correlation Structure: corBrownian
##  Formula: ~1 
##  Parameter estimate(s):
## numeric(0)
## 
## Coefficients:
##                   Value  Std.Error   t-value p-value
## (Intercept) -0.17975456 0.14940859 -1.203107  0.2380
## Lnalt        0.05475373 0.02162382  2.532103  0.0166
## 
##  Correlation: 
##       (Intr)
## Lnalt -0.879
## 
## Standardized residuals:
##        Min         Q1        Med         Q3        Max 
## -1.4790545 -1.0293796 -0.6849539 -0.4535629  2.0893357 
## 
## Residual standard error: 0.1289145 
## Degrees of freedom: 33 total; 31 residual
```

# 9
*** Also, here I should make a plot that somehow shows the graphic representation of the relationships between occurance of atherosclerosis in birds vs the other factors such as body mass, altitude of flight and etc. *** 

***I am still debating what type of figure will be a good option for showing this relationship here***

[Figure of relationship of athero and other factors]

# References

Abegglen, L. M., Caulin, A. F., Chan, A., Lee, K., Robinson, R., Campbell, M. S., ... & Jensen, S. T. (2015). Potential mechanisms for cancer resistance in elephants and comparative cellular response to DNA damage in humans. Jama, 314(17), 1850-1860.
<br><br>
Blumstein, D. T., Buckner, J., Shah, S., Patel, S., Alfaro, M. E., & Natterson-Horowitz, B. (2015). The evolution of capture myopathy in hooved mammals: a model for human stress cardiomyopathy?. Evolution, medicine, and public health, 2015(1), 195-203.
<br><br>
Finlayson, R., Symons, C., & TW-Fiennes, R. N. (1962). Atherosclerosis: a comparative study. British medical journal, 1(5277), 501.




